In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Definimos fuente para los graficos
from matplotlib import rcParams
import matplotlib.font_manager as font_manager

font_dir = ['/home/daniufundar/Downloads/Archivo/']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

rcParams['font.family'] = 'Archivo'

fundar_colors = ['#E27124','#AFA36E', '#5D896F','#7BB5C4', '#B3B3B3']

In [2]:
import platform

if platform.system() == "Windows":
    pathfolder = 'path_for_Windows'
elif platform.system() == "Linux":
    if platform.uname()[2] == "5.15.0-57-generic":
        pathfolder = '/home/daniufundar/Documents/Fundar/indice-desigualdad-genero-fundar/'
    elif platform.uname()[2] == "version2":
        pathfolder = '/Users/danielarisaro/Documents/fundar/indice-desigualdad-genero/'
elif platform.system() == "Darwin":
    pathfolder = '/Users/danielarisaro/Documents/fundar/indice-desigualdad-genero/'
else:
    raise Exception("Unsupported operating system")


In [3]:
pathfolder = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathinput = pathfolder + 'data_input/'

filename_hogar = 'hogar_T'
filename_personas = 'personas_T'

list_anios = [20, 21]
list_trimestres = [1, 2, 3, 4]

In [4]:
for i, it in enumerate(list_trimestres):
    for j, janios in enumerate(list_anios):

        filename = filename_personas + str(it) + str(janios) + '.txt'
        df = pd.read_csv(pathinput + filename, delimiter=';', low_memory=False)

In [18]:
pathfolder_ENUT =  '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/data_input/ENUT_2021/Bases de datos/'
filename = 'enut2021_base.txt'
df_enut_base = pd.read_csv(pathfolder_ENUT + filename, sep='|')

np.array(df_enut_base.columns)

array(['ID', 'WHOG', 'WPER', 'REGION', 'N_MIEMBRO', 'BHCV01', 'BHCV02',
       'BHCV03', 'BHCV04', 'BHCV05', 'BHCV06', 'BHCV07', 'BHCV08',
       'BHCV09', 'BHCV10', 'BHCV11', 'BHHO01', 'BHHO02', 'BHHO03',
       'EDAD_SEL', 'GRUPO_EDAD_SEL', 'BHCH04_SEL', 'SEXO_SEL',
       'BHCH06_SEL', 'BHCH07_SEL', 'BHCH08_SEL', 'BHCH09_SEL',
       'BHCH10_01_SEL', 'BHCH10_02_SEL', 'BHCH10_99_SEL', 'BHCH11_SEL',
       'BHCH12_SEL', 'BHCH13_SEL', 'NIVEL_EDUCATIVO_SEL',
       'NIVEL_EDUCATIVO_AGRUPADO', 'BHCH14_01_SEL', 'BHCH14_02_SEL',
       'BHCH14_03_SEL', 'BHCH14_04_SEL', 'BHCH14_05_SEL', 'BHCH14_06_SEL',
       'BHCH14_07_SEL', 'BHCH14_99_SEL', 'BHCH15_SEL',
       'CANT_MIEMBROS_HOGAR', 'CANT_PERSONASHASTA13',
       'CANT_PERSONAS14A64', 'CANT_PERSONAS65YMAS',
       'CANT_DEMANDANTES_TOTAL', 'CANT_NODEMANDANTES_TOTAL',
       'CANT_DEMANDANTES_14A64', 'CANT_DEMANDANTES_65YMAS',
       'CANT_AYUDA_TOTAL', 'BHDC_SEL', 'BHDC01_01_SEL', 'BHDC01_02_SEL',
       'BHDC01_03_SEL', 'BHDC01_04_SEL'

In [10]:
len(df_enut_base)

14350

In [19]:
pathfolder_ENUT =  '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/data_input/ENUT_2021/Bases de datos/'
filename = 'enut2021_miembros.txt'
df_enut_miembros = pd.read_csv(pathfolder_ENUT + filename, sep='|')


In [20]:
df_enut_miembros

,ID,N_MIEMBRO,BHCH03,BHCH04,BHCH05,BHCH06,BHCH07,BHCH08,BHCH09,BHCH10_01,...,BHDC01_05,BHDC01_06,BHDC01_07,BHDC01_08,BHDC02_01,BHDC02_02,BHDC02_03,BHDC02_04,BHDC02_05,BHDC02_06
0,112610,1,42,1,2,4.0,6,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,112610,2,15,3,2,4.0,6,1,2.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,138176,1,35,1,1,3.0,6,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,192097,1,26,1,1,3.0,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,192097,2,31,2,2,4.0,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41020,198186,1,66,1,2,4.0,2,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41021,198186,2,64,2,1,3.0,2,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41022,108973,1,28,1,1,3.0,6,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41023,108973,2,9,3,1,NaN,6,1,1.0,5.0,...,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
